In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.insert(0, module_path)
import ccd.parameters
import ccd
#from ccd.main import fusion_image
import numpy as np
from datetime import datetime
from datetime import date
import time
from data_input import line
import matplotlib.pyplot as plt

In [ ]:
image_dir = '/Users/arthur.platel/Desktop/trial2' #Fusion_imperial' #YATSM/fusion_images/'


In [ ]:
import glob
result=line(image_dir,20)


In [ ]:
#def pixel_array(pixel_x,pixel_y, dict):
#   pixel=dict[(pixel_x,pixel_y)]
#   return np.array([pixel['dates'],pixel['blue'],pixel["green"], pixel['red'], pixel['nir'], pixel['ndvi'],pixel['qas']])

#data = pixel_array(150,150,result)
#dates,blues,greens, reds, nirs, ndvis, qas = data


In [ ]:
params = {'QA_BITPACKED': False,
              'QA_FILL': 255,
              'QA_CLEAR': 0,
              'QA_WATER': 1,
              'QA_SHADOW': 2,
              'QA_SNOW': 3,
              'QA_CLOUD': 4}
results = ccd.detect(dates, greens,blues, reds, nirs, ndvis, qas, params)

In [ ]:
from datetime import date

def plot(band,bands,results,dates):
    predicted_values = []
    prediction_dates = []
    break_dates = []
    start_dates = []
    mask = np.array(results['processing_mask'], dtype=bool)

    for num, result in enumerate(results['change_models']):
        days = np.arange(result['start_day'], result['end_day'] + 1)
        prediction_dates.append(days)
        break_dates.append(result['break_day'])
        start_dates.append(result['start_day'])
        
        intercept = result[band]['intercept']
        coef = result[band]['coefficients']
        
        predicted_values.append(intercept + coef[0] * days +
                                coef[1]*np.cos(days*1*2*np.pi/365.25) + coef[2]*np.sin(days*1*2*np.pi/365.25) +
                                coef[3]*np.cos(days*2*2*np.pi/365.25) + coef[4]*np.sin(days*2*2*np.pi/365.25) +
                                coef[5]*np.cos(days*3*2*np.pi/365.25) + coef[6]*np.sin(days*3*2*np.pi/365.25))
        
    plt.style.use('ggplot')
    fg = plt.figure(figsize=(16,9), dpi=300)
    if band=='ndvi':
        a1 = fg.add_subplot(2, 1, 1, xlim=(min(dates), max(dates)) )   
    elif band=='nir':
        a1 = fg.add_subplot(2, 1, 1, xlim=(min(dates), max(dates)), ylim=(0, 5000))
    else:
        a1 = fg.add_subplot(2, 1, 1, xlim=(min(dates), max(dates)), ylim=(0, 800))

    for _preddate, _predvalue in zip(prediction_dates, predicted_values):
        a1.plot(_preddate, _predvalue, 'orange', linewidth=1)

    # fg = plt.figure(figsize=(16,9), dpi=300)
    # a1 = fg.add_subplot(2, 1, 1, xlim=(min(dates), max(dates), ylim=(-5000, 5000))

    # Predicted curves
    for _preddate, _predvalue in zip(prediction_dates, predicted_values):
        a1.plot(_preddate, _predvalue, 'orange', linewidth=2)

    a1.plot(dates[mask], bands[mask], 'g+') # Observed values
    a1.plot(dates[~mask], bands[~mask], 'k+') # Observed values masked out
    for b in break_dates: a1.axvline(b)
    for s in start_dates: a1.axvline(s, color='r')
    plt.title(band)
    return a1

    
bands = ["blue","green","red","nir","ndvi"]
for k in range(len(bands)):
    plot(bands[k],data[k+1],results,dates)

In [ ]:
print(results)
# final=fusion_image(image_dir,bands)
# print(final)

In [ ]:
print()
# def pixel_array(pixel_x,pixel_y,dict)
#     return nparray.dict[]

In [ ]:
# def create_raster(day,band, type,dict,shape):
#     band_array=np.empty(shape)
#     for x in range(shape[0]):
#         for y in range(shape[1]):
#             print("pixel {},{}".format(x,y))
#             start_time = time.time()
#             for num in dict[x,y]:
#                 print("RMSE: {}: ".format(num["blue"]["rmse"]))
#             pixel_time=time.time()-start_time

# create_raster(778990,"blue", "rmse", final, (2,2))